## Run inference global fit 

In [1]:
""" 
IMPORTS
"""
import sys
import os
import autograd.numpy as np
from glm_hmm_utils import load_cluster_arr, load_session_fold_lookup, \
    load_data, create_violation_mask, launch_glm_hmm_job

In [2]:
""" 
PARAMETERS
"""
D = 1  # data (observations) dimension
C = 2  # number of output types/categories
N_em_iters = 300  # number of EM iterations

USE_CLUSTER = False

num_folds = 5
global_fit = True
# perform mle => set transition_alpha to 1
transition_alpha = 1
prior_sigma = 100

bin_num = 10

In [3]:
"""
DIRECTORIES
"""
data_dir = '../../data/ibl/data_for_cluster/'
data_dir = '/home/ines/repositories/representation_learning_variability/DATA/GLMHMM/'
#data_dir = '/Users/ineslaranjeira/Documents/Repositories/representation_learning_variability/DATA/GLMHMM/'

"""
Create folders to save processed data
"""
# Create directories for saving data:
processed_ibl_data_path = data_dir + "data_for_cluster/" + "data_by_bin" + str(bin_num) + "global_normalization/"

# Create directory for results:
results_dir = '../../results/ibl_global_fit/'

In [4]:
#  In order to facilitate parallelization of jobs, create a job array that
#  can be used on e.g. a cluster
import numpy as np

K_vals = [2, 3, 4, 5]
K_vals = [3]
num_folds = 5
N_initializations = 10

cluster_job_arr = []
for K in K_vals:
    for i in range(num_folds):
        for j in range(N_initializations):
            cluster_job_arr.append([K, i, j])
np.savez(processed_ibl_data_path + 'cluster_job_arr.npz',
            cluster_job_arr)

In [5]:

if USE_CLUSTER:
    z = int(sys.argv[1])
else:
    z = 0
        
# Load external files:
cluster_arr_file = processed_ibl_data_path + 'cluster_job_arr.npz'
# Load cluster array job parameters:
cluster_arr = load_cluster_arr(cluster_arr_file)


[K, fold, iter] = cluster_arr[z]


# Get all data

animal_file = processed_ibl_data_path + 'all_animals_concat.npz'
inpt, y, bin_data = load_data(animal_file)
bin_fold_lookup_table = load_session_fold_lookup(
    processed_ibl_data_path + 'all_animals_concat_session_fold_lookup.npz')

bin_values = np.unique(bin_data)
bin_values = [0, 5, 9]
# Loop through bins and fit a separate global GLM for each bin
for b, bin in enumerate(bin_values):
    
    for m, model in enumerate(range(len(cluster_arr))):
        
        [K, fold, iter] = cluster_arr[m]
            
        # Subset of trials of interest for bin
        bin_idx = np.where(bin_data == bin)
        bin_y = y[bin_idx]
        bin_bin_fold_lookup_table = bin_fold_lookup_table[bin_idx]
        bin_inpt = inpt[bin_idx]
        bin_bin_data = bin_data[bin_idx]  

        #  append a column of ones to inpt to represent the bias covariate:
        bin_inpt = np.hstack((bin_inpt, np.ones((len(bin_inpt),1))))
        bin_y = bin_y.astype('int')
        # Identify violations for exclusion:
        violation_idx = np.where(bin_y == -1)[0]
        nonviolation_idx, mask = create_violation_mask(violation_idx,
                                                        bin_inpt.shape[0])

        #  GLM weights to use to initialize GLM-HMM
        init_param_file = results_dir + 'GLM/bin_' + str(bin+1) + '_fold_' + str(
            fold) + '/variables_of_interest_iter_0.npz'

        # create save directory for this initialization/fold combination:
        save_directory = results_dir + '/GLM_HMM_K_' + str(
            K) + '/bin_' + str(bin+1) + '/fold_' + str(fold) + '/' 
        if not os.path.exists(save_directory):
            os.makedirs(save_directory)

        launch_glm_hmm_job(bin_inpt,
                            bin_y,
                            bin_bin_data,
                            mask,
                            bin_bin_fold_lookup_table,
                            K,
                            D,
                            C,
                            N_em_iters,
                            transition_alpha,
                            prior_sigma,
                            fold,
                            iter,
                            global_fit,
                            init_param_file,
                            save_directory)

Starting inference with K = 3; Fold = 0; Iter = 0
=== fitting GLM-HMM ========


  0%|          | 0/300 [00:00<?, ?it/s]